<a href="https://colab.research.google.com/github/devvats/Hand-Digit-Recognition/blob/master/HandDigit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv("drive/My Drive/Hand Digit/digit-recognizer/train.csv")
test = pd.read_csv("drive/My Drive/Hand Digit/digit-recognizer/test.csv")

In [0]:
print(train.shape)

(42000, 785)


In [0]:
print(test.shape)

(28000, 784)


In [0]:
x = np.array(train)
np.random.shuffle(x)

In [0]:
y = x[:,0]
x = x[:,1:]

In [0]:
print(y.shape)
print(x.shape)
x.resize(42000,28,28)
x = x[:,:,:,np.newaxis]
print(x.shape)

(42000,)
(42000, 1, 28, 28)
(42000, 28, 28, 1)


In [0]:
from keras.utils import Sequence,to_categorical

class Generator(Sequence):
  
    def __init__(self,X,Y,batch,n_classes):
        self.x=X
        self.y=Y
        self.batch=batch
        self.n_classes=n_classes
        self.cnt=0
    
    def __len__(self):
        print(np.array(self.x).shape)
        return int(np.floor(np.array(self.x).shape[0]) / self.batch)
  
  
    def __getitem__(self,index):
      
        X=self.x[index*self.batch:(index+1)*self.batch]
        Y=self.y[index*(self.batch):(index+1)*self.batch]
        X=np.array(X)
        Y=np.array(Y)
        return X,to_categorical(Y,self.n_classes)

In [0]:
Xtrain = x[:30000]
Ytrain = y[:30000]
Xval = x[30000:]
Yval = y[30000:]
train = Generator(Xtrain, Ytrain,32,10)
validation = Generator(Xval, Yval, 32, 10)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
np.random.seed(17)

model = Sequential()
model.add(Conv2D(filters=10, input_shape=(28,28,1), kernel_size=(4,4), strides=(2,2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))


model.add(Conv2D(filters=10, kernel_size=(10,10), strides=(1,1), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(filters=10, kernel_size=(20,20), strides=(2,2), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))
model.add(GlobalAveragePooling2D(data_format='channels_last'))

model.add(Dense(200, input_shape=(1*1*1,)))
model.add(Activation('relu'))


model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 14, 14, 10)        170       
_________________________________________________________________
activation_37 (Activation)   (None, 14, 14, 10)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 14, 14, 10)        10010     
_________________________________________________________________
activation_38 (Activation)   (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 7, 7, 10)          40010     
_________________________________________________________________
activation_39 (Activation)   (None, 7, 7, 10)          0         
__________

In [0]:
#model.load_weights("drive/My Drive/code marathon dataset/data/model.h5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
Process = model.fit_generator(generator=train,
                    validation_data=validation,
                    use_multiprocessing=True,
                    workers = 6,
                    epochs = 15,
                    verbose = 1,
                    shuffle = True
                             )

(12000, 28, 28, 1)
(30000, 28, 28, 1)
(12000, 28, 28, 1)
Epoch 1/15
(12000, 28, 28, 1)
(30000, 28, 28, 1)
937/937 [==============================] - 104s 111ms/step - loss: 0.3281 - acc: 0.8923 - val_loss: 0.0936 - val_acc: 0.9722
Epoch 2/15
937/937 [==============================] - 105s 112ms/step - loss: 0.0850 - acc: 0.9742 - val_loss: 0.1116 - val_acc: 0.9670
Epoch 3/15
937/937 [==============================] - 103s 110ms/step - loss: 0.0621 - acc: 0.9809 - val_loss: 0.1182 - val_acc: 0.9689
Epoch 4/15
937/937 [==============================] - 105s 112ms/step - loss: 0.0485 - acc: 0.9843 - val_loss: 0.0939 - val_acc: 0.9742
Epoch 5/15
937/937 [==============================] - 106s 113ms/step - loss: 0.0414 - acc: 0.9866 - val_loss: 0.0849 - val_acc: 0.9777
Epoch 6/15
937/937 [==============================] - 105s 112ms/step - loss: 0.0341 - acc: 0.9895 - val_loss: 0.1181 - val_acc: 0.9693
Epoch 7/15
937/937 [==============================] - 105s 112ms/step - loss: 0.0358 - ac

In [0]:
model.save_weights("drive/My Drive/Hand Digit/model.h5")

In [0]:
xtest = np.array(test)
xtest.resize(28000,28,28,1)


In [0]:
ytest = model.predict(xtest)

[[5.2996816e-26 1.6354679e-21 1.0000000e+00 ... 8.3523968e-20
  7.2679064e-28 3.2685525e-34]
 [1.0000000e+00 1.5064534e-18 7.7462870e-17 ... 4.3489768e-22
  2.2919925e-16 1.9601525e-17]
 [2.1791971e-06 7.8773354e-08 5.3979754e-02 ... 4.3360526e-03
  6.5209819e-03 9.3491417e-01]
 ...
 [2.2190014e-32 4.5318796e-27 7.1339260e-25 ... 2.0647346e-24
  1.1014197e-22 5.4745565e-23]
 [2.5272637e-09 3.3586372e-07 9.7114150e-07 ... 1.8134614e-02
  1.4155058e-04 8.8637054e-01]
 [4.1514229e-29 6.0054891e-25 1.0000000e+00 ... 4.4287995e-23
  1.9391444e-30 2.9228631e-37]]


In [0]:
ytestt = np.argmax(ytest,axis =1)

In [0]:
print(ytestt)
num= np.array(range(1,28001))

[2 0 9 ... 3 9 2]


In [0]:
#ytestt.resize(28000,1)
#num.resize(28000,1)
#num = np.hstack([num,ytestt])

In [0]:
dff = pd.DataFrame({'ImageId':num,'Label':ytestt})
dff.to_csv('drive/My Drive/Hand Digit/digit-recognizer/res.csv',index = False)